In [1]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.9 MB/s eta 0:00:00


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.nn import DataParallel

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/My Drive/Colab Projects/BioDiversity Project

/content/drive/My Drive/Colab Projects/BioDiversity Project


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "dipro7/biodiversity_llm_v2"  # Replace with your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

OSError: ignored

In [5]:
!pip install peft

In [6]:
!pip install accelerate

In [7]:
!pip install sentencepiece

In [8]:
import torch

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [10]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "abhishek/llama-2-7b-hf-small-shards"
adapters_name = "dipro7/biodiversity_llm_v3"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tok = LlamaTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

Starting to load the model abhishek/llama-2-7b-hf-small-shards into memory


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Successfully loaded the model abhishek/llama-2-7b-hf-small-shards into memory


In [12]:
# Input text
input_text = "Axis porcinus size"

# Tokenize the input text
input_ids = tok.encode(input_text, return_tensors="pt")

# Move the input_ids tensor to the same device as the model (CPU)
input_ids = input_ids.to(device)

# Generate text based on the input
output = m.generate(input_ids, max_length=500, num_return_sequences=1)

# Decode the generated output
generated_text = tok.decode(output[0], skip_special_tokens=False)
print(generated_text)

<s> Axis porcinus size: 100-120 cm; weight: 10-15 kg

### Morphology:
The Axis deer is a medium-sized deer with a body length ranging from about 100 to 120 cm (39 to 47 inches) and a shoulder height of around 70 to 90 cm (28 to 35 inches). They have a slender build with a long neck and a distinctive white rump patch. Adult males have antlers that are typically larger and more elaborate than those of females.

### Behavior:
Axis deer are primarily diurnal, meaning they are most active during the day. They are herbivores, feeding on a variety of grasses, leaves, and other plant materials. They are known for their agility and can move quickly through dense vegetation.

### Distribution:
The Axis deer has a wide distribution across Asia, including countries such as India, Nepal, Sri Lanka, and parts of Southeast Asia. They inhabit a range of habitats, from forests to grasslands.

### Habitat:
Axis deer are adaptable to various environments and can be found in a range of habitats, including

In [ ]:
# Input text
input_context = '''
SpeciesName: Tupaia belangeri

Overview:
'''

# Tokenize the input text
input_ids = tok.encode(input_context, return_tensors="pt")

# Move the input_ids tensor to the same device as the model (CPU)
input_ids = input_ids.to(device)

# Generate text based on the input
output = m.generate(input_ids, max_length=500, temperature=0.4, num_return_sequences=1)

# Decode the generated output
generated_text = tok.decode(output[0], skip_special_tokens=False)
print(generated_text)

<s> 
SpeciesName: Tupaia belangeri

Overview: 
The Belanger's Treeshrew (Tupaia belangeri) is a species of treeshrew found in the forests of Southeast Asia. It is known for its arboreal lifestyle and its ability to climb trees with ease.
### Size:
Belanger's Treeshrews have a body length ranging from about 15 to 20 cm (5.9 to 7.9 inches), and their tail can measure around 15 to 20 cm (5.9 to 7.9 inches). They generally weigh between 100 to 150 grams (3.5 to 5.3 ounces).
### Morphology:
These shrews have a slender body with a pointed snout and large eyes. They have a dark brown to black fur coat with a lighter underside. Their feet are adapted for climbing trees and their claws are sharp for gripping branches.
### Behavior:
Belanger's Treeshrews are primarily arboreal, spending most of their time in trees. They are known for their agility and ability to climb trees with ease. They are primarily nocturnal, being most active during the night.
### Distribution:
The range of Belanger's Tree

In [ ]:
save_model_directory = "/content/drive/MyDrive/Colab Projects/BioDiversity Project/Saved_LLM_Model-v1"

import os
os.makedirs(save_model_directory, exist_ok=True)

m.save_pretrained(save_model_directory)
tokenizer.save_pretrained(save_model_directory)

print(f"Model and tokenizer saved to {save_model_directory}")

NameError: ignored

In [13]:
save_model_directory = "/content/drive/MyDrive/Colab Projects/BioDiversity Project/Saved_LLM_Model_V3"

model_name = "abhishek/llama-2-7b-hf-small-shards"
adapters_name = "dipro7/biodiversity_llm_v3"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()

# Load the tokenizer
tok = LlamaTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

import os
os.makedirs(save_model_directory, exist_ok=True)

m.save_pretrained(save_model_directory)
tok.save_pretrained(save_model_directory)

print(f"Model and tokenizer saved to {save_model_directory}")

Starting to load the model abhishek/llama-2-7b-hf-small-shards into memory


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:508: UserWarning: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration. This warning will be raised to an exception in v4.34.

Thrown during validation:
`do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Model and tokenizer saved to /content/drive/MyDrive/Colab Projects/BioDiversity Project/Saved_LLM_Model_V3


In [14]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:
huggingface_hub push save_model_directory

SyntaxError: ignored

In [20]:
pt_model = AutoModelForCausalLM.from_pretrained(save_model_directory, from_tf=True)
pt_model.save_pretrained(save_model_directory)

NameError: ignored

In [19]:
tf_model = AutoModelForCausalLM.from_pretrained(save_model_directory, from_pt=True)

NameError: ignored